<a href="https://colab.research.google.com/github/NikhilGarge/personalized-medicine-selector/blob/main/Personalized_Treatment_genomics_T2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# Personalized Treatment Selection for T2D with Deep Learning on Genomic Data
# Author: Nikhil Garge
# Date: 2025-06-17
# Description: This code implements a deep learning model for personalized treatment selection based on SNP and clinical data.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [90]:
def simulate_snp_matrix(n_patients, n_snps, maf_min=0.1, maf_max=0.5, random_state=None):
    rng = np.random.default_rng(random_state)
    maf_vec = rng.uniform(maf_min, maf_max, size=n_snps)
    # Each SNP column is generated independently
    snp_matrix = np.column_stack([rng.binomial(2, maf, size=n_patients) for maf in maf_vec])
    snp_df = pd.DataFrame(snp_matrix, columns=[f"SNP{i+1}" for i in range(n_snps)])
    return snp_df

def simulate_patient_data(
    n_patients,
    n_snps,
    snp_df,
    n_causal_snps=20,
    n_interacting_snps=10,
    n_epistatic_pairs=5,
    treatment_effects=None,
    random_state=None
):
    rng = np.random.default_rng(random_state)
    if treatment_effects is None:
        treatment_effects = {'T1': 0, 'T2': -0.3, 'T3': -0.6, 'T4': -0.9}
    treatments = list(treatment_effects.keys())
    n_treatments = len(treatments)
    # Assign treatments randomly
    treatment_assignment = rng.choice(treatments, size=n_patients, replace=True)
    # Select causal SNPs and assign effects
    causal_indices = rng.choice(n_snps, size=n_causal_snps, replace=False)
    causal_treatments = rng.choice(treatments, size=n_causal_snps, replace=True)
    causal_effects = rng.normal(loc=-0.6, scale=0.2, size=n_causal_snps)
    baseline_hba1c = 8
    hba1c = baseline_hba1c + rng.normal(0, 0.5, size=n_patients)
    snp_matrix = snp_df.values

    # Add SNP-treatment interactions
    for i in range(n_causal_snps):
        idx = np.where(treatment_assignment == causal_treatments[i])[0]
        hba1c[idx] += snp_matrix[idx, causal_indices[i]] * causal_effects[i]

    # Add main treatment effects
    for t in treatments:
        idx = np.where(treatment_assignment == t)[0]
        hba1c[idx] += treatment_effects[t]

    # Add epistatic effects (SNP-SNP interaction)
    if n_epistatic_pairs > 0:
        epistatic_indices = rng.choice(n_snps, size=2 * n_epistatic_pairs, replace=False).reshape(n_epistatic_pairs, 2)
        for j in range(n_epistatic_pairs):
            snp1, snp2 = epistatic_indices[j]
            epi_effect = rng.normal(loc=-0.4, scale=0.15)
            hba1c += (snp_matrix[:, snp1] * snp_matrix[:, snp2]) * epi_effect

    # Add random noise
    hba1c += rng.normal(0, 0.3, size=n_patients)

    # Assemble DataFrame
    geno_data = pd.DataFrame({
        'PatientID': np.arange(1, n_patients + 1),
        'Treatment': treatment_assignment,
        'HbA1c': hba1c
    })
    geno_data = pd.concat([geno_data, snp_df], axis=1)
    geno_data['Treatment'] = pd.Categorical(geno_data['Treatment'], categories=treatments)

    return geno_data

In [91]:
# Set parameters
n_patients = 10000
n_snps = 2000

# Simulate SNP matrix
snp_df = simulate_snp_matrix(n_patients, n_snps, maf_min=0.1, maf_max=0.5, random_state=42)

# Simulate patient data
geno_data = simulate_patient_data(
    n_patients=n_patients,
    n_snps=n_snps,
    snp_df=snp_df,
    n_causal_snps=20,
    n_interacting_snps=10,
    n_epistatic_pairs=5,
    treatment_effects={'T1': 0, 'T2': -0.3, 'T3': -0.6, 'T4': -0.9},
    random_state=123
)

In [92]:
# # prompt: read csv file and make it into pandas data frame
# geno_data = pd.read_csv('Simulated_Geno_data_10Kpatients.csv')
# geno_data.head(10)
geno_data.head(10)

,PatientID,Treatment,HbA1c,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,...,SNP1991,SNP1992,SNP1993,SNP1994,SNP1995,SNP1996,SNP1997,SNP1998,SNP1999,SNP2000
0,1,T1,6.421560,2,0,1,0,0,0,1,...,0,1,0,0,0,2,0,0,2,1
1,2,T3,4.245592,1,0,1,1,0,2,2,...,1,0,1,0,1,0,1,1,0,0
2,3,T3,5.860412,2,0,1,0,0,0,0,...,0,0,1,0,1,1,0,1,1,0
3,4,T1,5.278980,1,2,0,2,0,1,0,...,0,1,1,0,0,1,0,0,0,0
4,5,T4,5.983924,0,0,1,1,0,1,0,...,1,1,1,1,1,2,0,0,0,1
5,6,T1,3.770834,2,0,1,0,0,1,0,...,1,2,0,0,1,0,0,0,0,1
6,7,T2,5.024877,1,0,1,2,1,1,0,...,0,1,0,2,1,1,0,0,2,1
7,8,T1,4.680315,0,1,2,1,1,1,0,...,1,2,1,1,0,2,0,0,0,1
8,9,T2,5.219656,0,1,1,0,0,2,2,...,0,0,0,0,0,0,1,0,1,0
9,10,T1,6.251204,1,0,2,1,0,0,0,...,1,1,1,0,0,1,0,1,2,1


In [93]:
snp_cols = [col for col in geno_data.columns if col.startswith('SNP')]
X_snp = geno_data[snp_cols].values.astype(np.float32)

# Create a mapping for treatment categories to integers
# You can dynamically generate this mapping if you have many treatments
treatment_mapping = {'T1': 0, 'T2': 1, 'T3': 2, 'T4': 3} # Add more as needed

# Map the 'Treatment' column using the defined mapping and convert to int32
X_treatment = geno_data['Treatment'].map(treatment_mapping).values.astype(np.int32)

y = geno_data['HbA1c'].values.astype(np.float32)

In [94]:
# Scale SNP data
scaler = StandardScaler()
X_snp = scaler.fit_transform(X_snp)

In [95]:
from tensorflow.keras.regularizers import l2

# Model definition
def create_treatment_model(snp_dim=2000, n_treatments=4):
    snp_input = Input(shape=(snp_dim,), name='snp_input')
    x = BatchNormalization()(snp_input)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.15))(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.15))(x)
    x = Dropout(0.3)(x)  # Increased dropout

    treatment_input = Input(shape=(1,), name='treatment_input')
    treatment_embed = Embedding(n_treatments, 4)(treatment_input)
    treatment_embed = Flatten()(treatment_embed)

    combined = Concatenate()([x, treatment_embed])
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.15))(combined)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='linear')(x)

    model = Model(inputs=[snp_input, treatment_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

model = create_treatment_model(snp_dim=X_snp.shape[1], n_treatments=4)

In [96]:
# prompt: split the data into train validation and test set with 75%, 15%, 10% split. Only Use validation set for fine tuning the model (like it was done in the code). But also give me the final accuracy on training and test sets using mae and R-squared metric.

from sklearn.metrics import mean_absolute_error, r2_score

# Split data into train, validation, and test sets (75%, 15%, 10%)
X_snp_train, X_snp_temp, X_treatment_train, X_treatment_temp, y_train, y_temp = train_test_split(
    X_snp, X_treatment, y, test_size=0.25, random_state=42
)

X_snp_val, X_snp_test, X_treatment_val, X_treatment_test, y_val, y_test = train_test_split(
    X_snp_temp, X_treatment_temp, y_temp, test_size=0.4, random_state=42 # 0.4 * 0.25 = 0.1 for test, 0.6 * 0.25 = 0.15 for validation
)

# Model training (using the existing training and validation sets)
model.fit(
    [X_snp_train, X_treatment_train],
    y_train,
    batch_size=64,
    epochs=100,
    validation_data=([X_snp_val, X_treatment_val], y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)]
)

# Evaluate on the training set
y_train_pred = model.predict([X_snp_train, X_treatment_train])
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print(f"Training Set Evaluation:")
print(f"  MAE: {mae_train:.4f}")
print(f"  R-squared: {r2_train:.4f}")

# Evaluate on the test set
y_test_pred = model.predict([X_snp_test, X_treatment_test])
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"\nTest Set Evaluation:")
print(f"  MAE: {mae_test:.4f}")
print(f"  R-squared: {r2_test:.4f}")


Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 95.1566 - mae: 2.1544 - val_loss: 35.9462 - val_mae: 1.2421
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 27.9031 - mae: 1.2278 - val_loss: 13.2100 - val_mae: 1.1308
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 11.1551 - mae: 1.1073 - val_loss: 6.3060 - val_mae: 1.0215
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 5.5039 - mae: 1.0015 - val_loss: 3.7406 - val_mae: 0.9668
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 3.5865 - mae: 1.0037 - val_loss: 2.8996 - val_mae: 0.9968
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 2.8165 - mae: 0.9992 - val_loss: 2.4162 - val_mae: 0.9541
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 2.4771 - mae: 0.9868 - val_loss: 2.1683 - val_mae: 0.9371
Epoch 8/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 2.3197 - mae: 1.0055 - val_loss: 2.0607 - val_mae: 0.9647
Epoch 9/100
118/118 ━━━━━━━━━━━━━━━

In [97]:
# Personalized treatment recommendation function
def recommend_best_treatment(model, scaler, patient_snp_row):
    # patient_snp_row: 1D array of 2000 SNPs (not scaled)
    patient_snp_scaled = scaler.transform(patient_snp_row.reshape(1, -1))
    treatments = np.arange(4).reshape(-1, 1)
    snp_batch = np.repeat(patient_snp_scaled, 4, axis=0)
    preds = model.predict([snp_batch, treatments], verbose=0).flatten()
    best_treatment = np.argmin(preds)
    return best_treatment, preds

# Example usage for a new patient:
new_patient_snp = geno_data.iloc[10][snp_cols].values
best_treatment, all_preds = recommend_best_treatment(model, scaler, new_patient_snp)
print(f"Recommended treatment: {best_treatment}, Predicted outcomes: {all_preds}")

Recommended treatment: 3, Predicted outcomes: [5.702407  5.7153587 5.205474  4.1559486]


In [98]:
# prompt: get the "best_treatment" and "all_preds" for all the patients in geno_data and give me the summary of output - how many were recommended for each treatment

# Inverse map treatment integers back to names for summary
inverse_treatment_mapping = {v: k for k, v in treatment_mapping.items()}

# Apply the recommendation function to all patients in the original geno_data
best_treatments = []
all_patients_preds = []

# Select a sample of 1000 rows from the original geno_data
sample_size = 3000
geno_data_sample = geno_data.sample(n=sample_size, random_state=42) # Using random_state for reproducibility

# Assuming geno_data contains all patients you want to evaluate
for index, row in geno_data_sample.iterrows():
    #print(index)
    patient_snp_row = row[snp_cols].values
    best_treatment_idx, preds = recommend_best_treatment(model, scaler, patient_snp_row)
    best_treatments.append(best_treatment_idx)
    all_patients_preds.append(preds)

# Convert the list of best treatments (indices) to a Pandas Series
recommended_treatments_series = pd.Series(best_treatments)

# Summarize the recommendations
recommendation_summary = recommended_treatments_series.value_counts().sort_index()

print("\nSummary of Recommended Treatments:")
for treatment_idx, count in recommendation_summary.items():
    treatment_name = inverse_treatment_mapping[treatment_idx]
    print(f"Treatment {treatment_name}: {count} patients")

# You can also store the best treatments and all predictions if needed
geno_data_sample['best_treatment_index'] = best_treatments
geno_data_sample['best_treatment'] = geno_data_sample['best_treatment_index'].map(inverse_treatment_mapping)
# Note: Storing all_patients_preds directly in the DataFrame might be complex due to varying lengths.
# You could store them as a list or in a separate structure if required for further analysis.
# For demonstration, let's just show the first few predictions.
# print("\nFirst 5 Patients' Predictions:")
# for i in range(min(5, len(all_patients_preds))):
#     print(f"Patient {i}: {all_patients_preds[i]}")


Summary of Recommended Treatments:
Treatment T4: 3000 patients


In [99]:
geno_data_sample.head()

,PatientID,Treatment,HbA1c,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,...,SNP1993,SNP1994,SNP1995,SNP1996,SNP1997,SNP1998,SNP1999,SNP2000,best_treatment_index,best_treatment
6252,6253,T2,3.397140,1,1,0,1,0,1,1,...,1,0,1,1,1,1,0,0,3,T4
4684,4685,T4,4.225944,0,0,0,2,0,1,0,...,1,0,0,1,0,0,1,0,3,T4
1731,1732,T2,4.385160,2,0,0,1,1,2,0,...,0,0,1,1,0,0,1,1,3,T4
4742,4743,T1,4.248011,0,1,0,1,0,2,0,...,1,0,0,1,0,1,0,0,3,T4
4521,4522,T2,6.920427,1,0,1,1,0,1,1,...,0,0,0,1,0,0,2,2,3,T4
